# Обработка данных

**Разделы:**
- [Импортируем библиотеки и загрузим данные](#импортируем-библиотеки-и-загрузим-данные)
- [Анализ и очистка данных](#анализ-и-очистка-данных)
- [Feature Enginnering](#Feature-Enginnering)
    - [Первичная-обработка](#первичная-обработка)
    - [Кодирование категориальных признаков](#кодирование-категориальных-признаков)
    - [Удаление ненужных столбцов](#удаление-ненужных-столбцов)
- [Разделение и сохранение данных](#разделение-и-сохранение-данных)

## Импортируем библиотеки и загрузим данные

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import sys

from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

In [10]:
# Загрузка данных из data/raw/
data_path = os.path.join('..', 'data', 'raw', 'Clean_Dataset.csv')
df = pd.read_csv(data_path)

## Анализ и очистка данных

In [11]:
print("🔍 Проверка пропусков:")
print(df.isnull().sum())

🔍 Проверка пропусков:
Unnamed: 0          0
airline             0
flight              0
source_city         0
departure_time      0
stops               0
arrival_time        0
destination_city    0
class               0
duration            0
days_left           0
price               0
dtype: int64


Пропусков нет, очень хорошо

In [12]:
print("\n📈 Статистика числовых признаков:")
print(df[['duration', 'days_left', 'price']].describe())


📈 Статистика числовых признаков:
            duration      days_left          price
count  300153.000000  300153.000000  300153.000000
mean       12.221021      26.004751   20889.660523
std         7.191997      13.561004   22697.767366
min         0.830000       1.000000    1105.000000
25%         6.830000      15.000000    4783.000000
50%        11.250000      26.000000    7425.000000
75%        16.170000      38.000000   42521.000000
max        49.830000      49.000000  123071.000000


Давайте удалим перелёты <1 часа... Это странно...

In [13]:
print(f"Количество данных: {df.shape[0]} (из них {df[df['duration'] < 1].shape[0]} это перелёты меньше 1 часа)")
df = df[df['duration'] >= 1]
print(f"Изменённое количество данных: {df.shape[0]}")

Количество данных: 300153 (из них 130 это перелёты меньше 1 часа)
Изменённое количество данных: 300023


## Feature Enginnering

Что мы будем делать:

1. Первичная обработка
   * `'stops encoding'`: {"zero":0, "one":1, "two_or_more":2}
        * Сохранится естестыенный порядок и логичность изменения
   * `'flight'` → `'flight_tag'` + `'flight_value'`
        * Таким образом мы разделим комплексный признак и будем получать информацию о авиакомпании и номер рейса.
   * `'route'` = `'source_city'` + "_" + `'destination_city'`
        * Сможем учитывать специфику конкретных направлений(к примеру Делли - Мумбаи может быть не равно Мумбаи - Делли).
        * Это позволит нам улавливать популярность маршрутов.
   * `'time_interval'` = `'departure_time'` + "_" + `'arrival_time'`
        * Комбинация времени позволит нам улавливать тонкости по времени перелёта.
        * Учитывает *удобства* путешествия

2. Кодирование категориальных признаков
   * `OneHotEncoder`: [`'airline'`, `'class'`, `'flight_tag'`, `'route'`, `'time_interval'`]
        * Кодировщик выбран с этими признаками так как у всех них нет определённой иерархии.

3. Масштабирование числовых признаков
   * `MinMaxScaler`: [`'duration'`, `'flight_value'`, `'days_left'`]
        * приводим всё к диапазону [0,1]
   * Логарифмирование цены: price_log = np.log1p(price)
        * Нормализует распределение
        * Уменьшит влияние выбросов
        * ВАЖНО для *линейных моделей*


In [14]:
# создадим копию данных для преобразований
df_processed = df.copy()

### Первичная обработка

Преобразовываем `'stops'` в `'stops_encoded'`

In [16]:
stops_dict = {"zero": 0, "one": 1, "two_or_more": 2}
df_processed['stops_encoded'] = df_processed['stops'].map(stops_dict)
print(f"Уникальные значения: {df_processed['stops_encoded'].unique()}")

Уникальные значения: [0 1 2]


Разделяем flight

In [17]:
df_processed['flight_tag'] = df_processed['flight'].str[:2]
df_processed['flight_value'] = df_processed['flight'].str[2:].astype(int)
print(f"flight_tag: {df_processed['flight_tag'].unique()}")
print(f"flight_value диапазон: {df_processed['flight_value'].min()} - {df_processed['flight_value'].max()}")

flight_tag: ['SG' 'I5' 'UK' 'G8' '6E' 'AI']
flight_value диапазон: -9991 - -101


Создаем route

In [19]:
df_processed['route'] = df_processed['source_city'] + "_" + df_processed['destination_city']
print(f"Уникальных маршрутов: {df_processed['route'].nunique()}")

Уникальных маршрутов: 30


Создаем time_interval

In [20]:
df_processed['time_interval'] = df_processed['departure_time'] + "_" + df_processed['arrival_time']
print(f"Уникальных интервалов: {df_processed['time_interval'].nunique()}")

Уникальных интервалов: 36


### Кодирование категориальных признаков

Используем `OneHotEncoder`

In [21]:
onehot_columns = ['airline', 'class', 'flight_tag', 'route', 'time_interval']
print(f"OneHot кодирование для: {onehot_columns}")

onehot_encoder = OneHotEncoder(sparse_output=False, drop='first')
onehot_encoded = onehot_encoder.fit_transform(df_processed[onehot_columns])

# создаем DataFrame с one-hot признаками
onehot_feature_names = onehot_encoder.get_feature_names_out(onehot_columns)
df_onehot = pd.DataFrame(onehot_encoded, columns=onehot_feature_names, index=df_processed.index)

# удаляем исходные колонки и добавляем one-hot
df_processed = df_processed.drop(onehot_columns, axis=1)
df_processed = pd.concat([df_processed, df_onehot], axis=1)

print(f"Добавлено {len(onehot_feature_names)} новых признаков")
print(f"Размер новых данных: {df_processed.shape}")

OneHot кодирование для: ['airline', 'class', 'flight_tag', 'route', 'time_interval']
Добавлено 75 новых признаков
Размер новых данных: (300023, 87)


### Масштабирование числовых признаков

In [28]:
numeric_columns = ['duration', 'flight_value', 'days_left']
print(f"Масштабируем: {numeric_columns}")
scaler = MinMaxScaler()
df_processed[numeric_columns] = scaler.fit_transform(df_processed[numeric_columns])

print(f"\nМасштабирование завершено. Диапазоны:")
for col in numeric_columns:
    print(f"  {col:20}: {df_processed[col].min():.1f} - {df_processed[col].max():.1f}")

Масштабируем: ['duration', 'flight_value', 'days_left']

Масштабирование завершено. Диапазоны:
  duration            : 0.0 - 1.0
  flight_value        : 0.0 - 1.0
  days_left           : 0.0 - 1.0


Добавим цену с логарифмированием

In [31]:
df_processed['price_log'] = np.log1p(df_processed['price'])
print(f"price_log создан. Диапазон: {df_processed['price_log'].min():.1f} - {df_processed['price_log'].max():.1f}")

price_log создан. Диапазон: 7.0 - 11.7


### Удаление ненужных столбцов

Сначало посмотрим какие у нас есть столбцы на данный момент:

In [35]:
columns = df_processed.columns.tolist()

for i in range(0, len(columns), 3):
    # форматируем вывод для лучшей читаемости
    row_columns = columns[i:i+3]
    formatted_columns = []
    
    for col in row_columns:
        # Обрезаем длинные названия для удобства чтения
        if len(col) > 25:
            col_display = col[:20] + "..."
        else:
            col_display = col
        formatted_columns.append(f"{col_display:<25}")
    
    print("  ".join(formatted_columns))

Unnamed: 0                 flight                     source_city              
departure_time             stops                      arrival_time             
destination_city           duration                   days_left                
price                      stops_encoded              flight_value             
airline_Air_India          airline_GO_FIRST           airline_Indigo           
airline_SpiceJet           airline_Vistara            class_Economy            
flight_tag_AI              flight_tag_G8              flight_tag_I5            
flight_tag_SG              flight_tag_UK              route_Bangalore_Delhi    
route_Bangalore_Hyderabad  route_Bangalore_Kolkata    route_Bangalore_Mumbai   
route_Chennai_Bangalore    route_Chennai_Delhi        route_Chennai_Hyderabad  
route_Chennai_Kolkata      route_Chennai_Mumbai       route_Delhi_Bangalore    
route_Delhi_Chennai        route_Delhi_Hyderabad      route_Delhi_Kolkata      
route_Delhi_Mumbai         route_Hyderab

Удалим исходные не нужные столбцы 

In [37]:
print(f"Кол. фич перед удалением: {len(df_processed.columns)}")

columns_to_drop = [
    'Unnamed: 0', 'flight', 'source_city', 'destination_city', 
    'departure_time', 'arrival_time', 'stops'
]
df_processed = df_processed.drop(columns_to_drop, axis=1)

print(f"Кол. фич после удаленгия: {len(df_processed.columns)}")

Кол. фич перед удалением: 88
Кол. фич после удаленгия: 81


## Разделение и сохранение данных

In [42]:
X = df_processed.drop(['price', 'price_log'], axis=1)
y = df_processed['price_log']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=df['class']
)

print(f"Итог раздения:\n")
print(f"   X_train: {X_train.shape}")
print(f"   X_test:  {X_test.shape}\n")
print(f"   y_train: {y_train.shape}")
print(f"   y_test:  {y_test.shape}")

Итог раздения:

   X_train: (240018, 79)
   X_test:  (60005, 79)

   y_train: (240018,)
   y_test:  (60005,)


In [43]:
os.makedirs('../data/processed', exist_ok=True)

X_train.to_csv('../data/processed/X_train.csv', index=False)
X_test.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)